## Run the entire code

In [20]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
%cd /content/drive/MyDrive/Colab/CICIDS2017 kitsune

/content/drive/MyDrive/Colab/CICIDS2017 kitsune


In [22]:
!pip install adversarial-robustness-toolbox

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
feature_mapping = "distribution"

#%% ART for TensorFlow v2 - Keras API
import warnings
warnings.filterwarnings('ignore')
import tensorflow as tf
tf.compat.v1.disable_eager_execution()
import numpy as np
import pandas as pd
import pickle as pkl
from sklearn.metrics import classification_report, precision_recall_fscore_support
import scipy.cluster.hierarchy as sch
from copy import deepcopy

from art.estimators.classification import KerasClassifier
from art.attacks.evasion import FastGradientMethod,ProjectedGradientDescent,DeepFool, CarliniL2Method
device = '/GPU:0'
if tf.__version__[0] != '2':
    raise ImportError('This notebook requires TensorFlow v2.')


#%% Load CICIDS2017 dataset
print('Loading CICIDS2017 dataset')
x_train = pd.read_csv('data/New_x_train.csv')
x_test = pd.read_csv('data/New_x_train.csv')
y_train = x_train.Label
y_test = x_test.Label

x_train = x_train.drop(columns=['Label'])
x_test = x_test.drop(columns=['Label'])
changeable = list(range(len(x_train.columns)))

x_train = x_train.values
x_test = x_test.values
y_train = y_train.values
y_test = y_test.values



Loading CICIDS2017 dataset


In [24]:
def feature_distribution(ind):
  feature_map = []
  for cluster in range(max_clust):
    feature_map.append(np.where(ind == cluster+1)[0].tolist())

  # Distribution
  distribute = []
  for index in range(len(feature_map)):
    if len(feature_map[index]) == 1:
      distribute.append((index, feature_map[index]))
  distribute.sort(reverse=True)
  for distrib in distribute:
    feature_map.pop(distrib[0])
    min(feature_map, key=len).append(distrib[1][0])
  return feature_map

### Cut

def feature_cut(ind):
  ordered = np.argsort(ind)
  feature_n = len(ordered) // max_clust
  rest = len(ordered) % max_clust
  feature_map = []
  i = 0
  while i < len(ordered):
    elem = feature_n if rest < 0 else feature_n + 1
    rest -= 1
    feature_map.append(ordered[i:i+elem].tolist())
    i += elem
  return feature_map

### Feature Mapping

max_clust = 5

corr = pd.DataFrame(x_train).corr().values
corr = np.nan_to_num(corr, nan=0)
dist = sch.distance.pdist(corr)
link = sch.linkage(dist, method='complete')
ind = sch.fcluster(link, max_clust, 'maxclust')

if feature_mapping == "cut":
  feature_map = feature_cut(ind)
elif feature_mapping == "distribution":
  feature_map = feature_distribution(ind)
else:
  print("Please set the feature_map variable.")

In [59]:
### Datasets

train_data = x_train.copy()
#y_train_ohe = pd.get_dummies(y_train_np)
datasets_train = []
for feature_list in feature_map:
  sub_data = []
  for row_index in range(len(train_data)):
    sub_data.append([train_data[row_index][feature] for feature in feature_list])
  datasets_train.append(sub_data)

for i in range(len(datasets_train)):
  datasets_train[i] = np.array(datasets_train[i])
#for i in range(len(datasets_train)):
  #with tf.device(device_name=device):
    #datasets_train[i] = tf.convert_to_tensor(datasets_train[i], np.float32)
#with tf.device(device_name=device):
  #y_train_ts = tf.convert_to_tensor(y_train_np, np.float32)
  #y_train_ohe = tf.convert_to_tensor(y_train_ohe, np.float32)

test_data = x_test.copy()
#y_test_ohe = pd.get_dummies(y_test_np)
datasets_test = []
for feature_list in feature_map:
  sub_data = []
  for row_index in range(len(test_data)):
    sub_data.append([test_data[row_index][feature] for feature in feature_list])
  datasets_test.append(sub_data)

for i in range(len(datasets_test)):
  datasets_test[i] = np.array(datasets_test[i])
#for i in range(len(datasets_test)):
  #with tf.device(device_name=device):
    #datasets_test[i] = tf.convert_to_tensor(datasets_test[i], np.float32)
#with tf.device(device_name=device):
  #y_test_ts = tf.convert_to_tensor(y_test_np, np.float32)
  #y_test_ohe = tf.convert_to_tensor(y_test_ohe, np.float32)

In [60]:
# Exporting the clean examples in a .pkl file
with open("clean_train.pkl", "wb") as f:
  pkl.dump(datasets_train, f)
with open("clean_test.pkl", "wb") as f:
  pkl.dump(datasets_test, f)

In [33]:
## Kitsutwo

### Design

output_number = 2
#eval_metric = 'categorical_accuracy'
#loss_fn = tf.keras.losses.CategoricalCrossentropy()
activ_out = 'softmax'
neurons_number = 256
lr = 0.01

ensemble_layer = []
for i in range(len(datasets_train)):
  ae = tf.keras.Sequential([
      tf.keras.layers.Dense(neurons_number, input_shape=(len(feature_map[i]),), activation="relu"),
      tf.keras.layers.Dense(neurons_number, activation="relu"),
      tf.keras.layers.Dense(neurons_number, activation="relu"),
      tf.keras.layers.Dense(output_number, activation=activ_out)
  ])
  ae.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  ensemble_layer.append(ae)

output_layer = tf.keras.Sequential([
      tf.keras.layers.Dense(neurons_number, input_shape=(len(ensemble_layer)*output_number,), activation="relu"),
      tf.keras.layers.Dense(neurons_number, activation="relu"),
      tf.keras.layers.Dense(neurons_number, activation="relu"),
      tf.keras.layers.Dense(1, activation='sigmoid')
])

output_layer.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=lr),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=['binary_accuracy']
)


In [34]:


### Ensemble Layer

for index in range(len(ensemble_layer)):
  best_value = 0.0
  model = ensemble_layer[index]
  model_file = "./Ensemble/{}".format(index)

  # Iterrating on the dataset
  for i in range(5):
    model.fit(x=datasets_train[index], y=y_train, epochs=10, batch_size=100, verbose=2)
    eval = model.evaluate(x=datasets_test[index], y=y_test, verbose=1)
    eval_value = eval[-1]
    if eval_value > best_value:
      best_value = eval_value
      tf.keras.models.save_model(model, model_file)

ensemble_layer = []
for i in range(len(feature_map)):
  ensemble_layer.append(tf.keras.models.load_model("./Ensemble/{}".format(i)))
  ensemble_layer[i].evaluate(x=datasets_test[i], y=y_test, verbose=2)



Train on 22731 samples
Epoch 1/10
22731/22731 - 2s - loss: 0.3950 - accuracy: 0.7900 - 2s/epoch - 69us/sample
Epoch 2/10
22731/22731 - 1s - loss: 0.3191 - accuracy: 0.8399 - 1s/epoch - 45us/sample
Epoch 3/10
22731/22731 - 1s - loss: 0.2990 - accuracy: 0.8472 - 1s/epoch - 47us/sample
Epoch 4/10
22731/22731 - 1s - loss: 0.2825 - accuracy: 0.8616 - 1s/epoch - 46us/sample
Epoch 5/10
22731/22731 - 1s - loss: 0.2715 - accuracy: 0.8696 - 1s/epoch - 45us/sample
Epoch 6/10
22731/22731 - 1s - loss: 0.2693 - accuracy: 0.8679 - 1s/epoch - 45us/sample
Epoch 7/10
22731/22731 - 1s - loss: 0.2593 - accuracy: 0.8756 - 1s/epoch - 45us/sample
Epoch 8/10
22731/22731 - 1s - loss: 0.2514 - accuracy: 0.8770 - 1s/epoch - 45us/sample
Epoch 9/10
22731/22731 - 1s - loss: 0.2421 - accuracy: 0.8770 - 1s/epoch - 46us/sample
Epoch 10/10
22731/22731 - 1s - loss: 0.2385 - accuracy: 0.8765 - 1s/epoch - 45us/sample
INFO:tensorflow:Assets written to: ./Ensemble/0/assets
Train on 22731 samples
Epoch 1/10
22731/22731 - 1s 

In [35]:
### Output Layer

predictions_train = []
predictions_test = []
output_number = 2
for i in range(len(ensemble_layer)):
  pred_tr = ensemble_layer[i].predict(datasets_train[i])
  pred_te = ensemble_layer[i].predict(datasets_test[i])
  for j in range(output_number):
    predictions_train.append([p[j] for p in pred_tr])
    predictions_test.append([p[j] for p in pred_te])

output_dataset_train =[[predictions_train[j][i] for j in range(len(predictions_train))] for i in range(len(predictions_train[0]))]
output_dataset_test = [[predictions_test[j][i] for j in range(len(predictions_test))] for i in range(len(predictions_test[0]))]
output_dataset_train = np.array(output_dataset_train)
output_dataset_test = np.array(output_dataset_test)
#with tf.device(device_name=device):
  #output_dataset_train = tf.convert_to_tensor(output_dataset_train, np.float32)
  #output_dataset_test = tf.convert_to_tensor(output_dataset_test, np.float32)

best_value = 0.0
model_file = "./Output/"

# Iterrating on the dataset
for i in range(5):
  output_layer.fit(x=output_dataset_train, y=y_train, epochs=10, batch_size=100, verbose=1)
  eval = output_layer.evaluate(x=output_dataset_test, y=y_test, verbose=0)
  eval_value = eval[-1]
  if eval_value > best_value:
    best_value = eval_value
    tf.keras.models.save_model(output_layer, model_file)

output_layer = tf.keras.models.load_model("./Output/")
output_layer.evaluate(x=output_dataset_test, y=y_test, verbose=1)



Train on 22731 samples
Epoch 1/10
22731/22731 [==============================] - 1s 64us/sample - loss: 0.1113 - binary_accuracy: 0.9616
Epoch 2/10
22731/22731 [==============================] - 1s 52us/sample - loss: 0.0897 - binary_accuracy: 0.9672
Epoch 3/10
22731/22731 [==============================] - 1s 52us/sample - loss: 0.0880 - binary_accuracy: 0.9666
Epoch 4/10
22731/22731 [==============================] - 1s 53us/sample - loss: 0.0844 - binary_accuracy: 0.9683
Epoch 5/10
22731/22731 [==============================] - 1s 54us/sample - loss: 0.0819 - binary_accuracy: 0.9690
Epoch 6/10
22731/22731 [==============================] - 1s 52us/sample - loss: 0.0775 - binary_accuracy: 0.9709
Epoch 7/10
22731/22731 [==============================] - 1s 53us/sample - loss: 0.0734 - binary_accuracy: 0.9735
Epoch 8/10
22731/22731 [==============================] - 1s 53us/sample - loss: 0.0671 - binary_accuracy: 0.9768
Epoch 9/10
22731/22731 [==============================] - 1s 52us

[0.24997800409124968, 0.4999780036074084, 0.33330889318384543, 0.9813031]

In [42]:
### Total

def get_accuracy(datasets, labels):
  ResultsMatrix= np.zeros(4)
  predictions = []
  for i in range(len(ensemble_layer)):
    pred = ensemble_layer[i].predict(datasets[i])
    for j in range(output_number):
      predictions.append([p[j] for p in pred])

  output_dataset = [[predictions[j][i] for j in range(len(predictions))] for i in range(len(predictions[0]))]
  output_dataset = np.array(output_dataset)
  #with tf.device(device_name=device):
    #output_dataset = tf.convert_to_tensor(output_dataset, np.float32)
  y_pred=output_layer.predict(output_dataset, verbose=2)
  pred_out=np.where(y_pred > 0.5, 1,0)
  print( classification_report(y_true=labels, y_pred=pred_out))
  ResultsMatrix=list( precision_recall_fscore_support(y_true=labels, y_pred=pred_out,average='weighted'))
  ResultsMatrix[3]= output_layer.evaluate(x=output_dataset, y=labels, verbose=0)[1]

  return ResultsMatrix


get_accuracy(datasets_test, y_test)

              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98     11365
         1.0       0.98      0.99      0.98     11366

    accuracy                           0.98     22731
   macro avg       0.98      0.98      0.98     22731
weighted avg       0.98      0.98      0.98     22731



[0.9813410708296506, 0.9813030662971273, 0.9813026935076935, 0.9813031]

In [43]:
#%%
## Adversarial Attacks

#### ART Classifiers

#lr = 0.1
#loss_fn = tf.keras.losses.CategoricalCrossentropy()
classifiers = []
for index in range(len(ensemble_layer)):
  shape = len(feature_map[index])
  model = ensemble_layer[index]
  classifiers.append(KerasClassifier(model=model, clip_values=(0, 1)))

#### non changeable features

to_attack = {i: [feature_map[i].index(j) for j in feature_map[i] if j not in list(set(feature_map[i]) & set(changeable))] for i in range(len(feature_map)) if list(set(feature_map[i]) & set(changeable)) != []}

In [44]:
#### FGSM

adversarial_examples_train = deepcopy(datasets_train)
adversarial_examples_test = deepcopy(datasets_test)
for index in list(to_attack.keys()):
  print('Crafting examples for classifier {}'.format(index))
  adversarial_crafter = FastGradientMethod(estimator=classifiers[index], eps=0.1)

  
  # Generating the adversarial examples
  adv_test = adversarial_crafter.generate(x=datasets_test[index])
  adv_train = adversarial_crafter.generate(x=datasets_train[index])

  # Removing changes made to unchangeable columns
  # for col in to_attack[index]:
  #   adv_test[:, col] = datasets_test[index][:, col]
  #   adv_train[:, col] = datasets_train[index][:, col]


  adversarial_examples_test[index] = deepcopy(adv_test)
  adversarial_examples_train[index]= deepcopy(adv_train)

# Saving adversarial examples
with open("New_FGSM_train.pkl", "wb") as f:
  pkl.dump(adversarial_examples_train, f)
with open("New_FGSM_test.pkl", "wb") as f:
  pkl.dump(adversarial_examples_test, f)

get_accuracy(adversarial_examples_test, y_test)

Crafting examples for classifier 0
Crafting examples for classifier 1
Crafting examples for classifier 2
Crafting examples for classifier 3
Crafting examples for classifier 4
              precision    recall  f1-score   support

         0.0       0.44      0.55      0.49     11365
         1.0       0.41      0.31      0.35     11366

    accuracy                           0.43     22731
   macro avg       0.42      0.43      0.42     22731
weighted avg       0.42      0.43      0.42     22731



[0.4249676176752102, 0.42910562667722496, 0.4211157970316891, 0.42910564]

In [45]:
#### PGD

adversarial_examples_train = deepcopy(datasets_train)
adversarial_examples_test = deepcopy(datasets_test)
for index in list(to_attack.keys()):
  print('Crafting examples for classifier {}'.format(index))
  adversarial_crafter = ProjectedGradientDescent(estimator=classifiers[index], eps=0.1,eps_step=0.01,verbose=False)

  # Generating the adversarial examples
  adv_test = adversarial_crafter.generate(x=datasets_test[index])
  adv_train = adversarial_crafter.generate(x=datasets_train[index])

  # # Removing changes made to unchangeable columns
  # for col in to_attack[index]:
  #   adv_test[:, col] = datasets_test[index][:, col]
  #   adv_train[:, col] = datasets_train[index][:, col]


  adversarial_examples_test[index] = deepcopy(adv_test)
  adversarial_examples_train[index]= deepcopy(adv_train)

# Saving adversarial examples
with open("New_PGD_train.pkl", "wb") as f:
  pkl.dump(adversarial_examples_train, f)
with open("New_PGD_test.pkl", "wb") as f:
  pkl.dump(adversarial_examples_test, f)

get_accuracy(adversarial_examples_test, y_test)

Crafting examples for classifier 0
Crafting examples for classifier 1
Crafting examples for classifier 2
Crafting examples for classifier 3
Crafting examples for classifier 4
              precision    recall  f1-score   support

         0.0       0.17      0.21      0.19     11365
         1.0       0.01      0.00      0.01     11366

    accuracy                           0.11     22731
   macro avg       0.09      0.11      0.10     22731
weighted avg       0.09      0.11      0.10     22731



[0.09041264369897206, 0.10787030926927983, 0.09826120745796456, 0.10787031]

In [46]:
#### DF

adversarial_examples_train = deepcopy(datasets_train)
adversarial_examples_test = deepcopy(datasets_test)
for index in list(to_attack.keys()):
  print('Crafting examples for classifier {}'.format(index))
  adversarial_crafter = DeepFool(classifier=classifiers[index])

  
 # Generating the adversarial examples
  adv_test = adversarial_crafter.generate(x=datasets_test[index])
  adv_train = adversarial_crafter.generate(x=datasets_train[index])

  # # Removing changes made to unchangeable columns
  # for col in to_attack[index]:
  #   adv_test[:, col] = datasets_test[index][:, col]
  #   adv_train[:, col] = datasets_train[index][:, col]


  adversarial_examples_test[index] = deepcopy(adv_test)
  adversarial_examples_train[index]= deepcopy(adv_train)

# Saving adversarial examples
with open("New_DF_train.pkl", "wb") as f:
  pkl.dump(adversarial_examples_train, f)
with open("New_DF_test.pkl", "wb") as f:
  pkl.dump(adversarial_examples_test, f)

get_accuracy(adversarial_examples_test, y_test)

Crafting examples for classifier 0


DeepFool:   0%|          | 0/22731 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/22731 [00:00<?, ?it/s]

Crafting examples for classifier 1


DeepFool:   0%|          | 0/22731 [00:00<?, ?it/s]

...

Crafting examples for classifier 4


DeepFool:   0%|          | 0/22731 [00:00<?, ?it/s]

DeepFool:   0%|          | 0/22731 [00:00<?, ?it/s]

              precision    recall  f1-score   support

         0.0       0.35      0.53      0.42     11365
         1.0       0.01      0.00      0.01     11366

    accuracy                           0.27     22731
   macro avg       0.18      0.27      0.21     22731
weighted avg       0.18      0.27      0.21     22731



[0.17806571660145087, 0.26716818441775547, 0.2126885485261639, 0.2671682]

In [ ]:

#### CW

adversarial_examples_train = deepcopy(datasets_train)
adversarial_examples_test = deepcopy(datasets_test)
for index in list(to_attack.keys()):
  print('Crafting examples for classifier {}'.format(index))
  adversarial_crafter = CarliniL2Method(classifier=classifiers[index],
                                        confidence=0.3,
                                        targeted=False,
                                        learning_rate=1,
                                        binary_search_steps=45,
                                        max_iter=10,
                                        initial_const=0.01,
                                        max_halving=5,
                                        max_doubling=5,
                                        batch_size=128,
                                        verbose=False)

  # Generating the adversarial examples
  adv_test = adversarial_crafter.generate(x=datasets_test[index])
  adv_train = adversarial_crafter.generate(x=datasets_train[index])

  # # Removing changes made to unchangeable columns
  # for col in to_attack[index]:
  #   adv_test[:, col] = datasets_test[index][:, col]
  #   adv_train[:, col] = datasets_train[index][:, col]


  adversarial_examples_test[index] = deepcopy(adv_test)
  adversarial_examples_train[index]= deepcopy(adv_train)

  # Saving adversarial examples
  with open("New_CW_train.pkl", "wb") as f:
    pkl.dump(adversarial_examples_train, f)
  with open("New_CW_test.pkl", "wb") as f:
    pkl.dump(adversarial_examples_test, f)

get_accuracy(adversarial_examples_test, y_test)

In [48]:
## Adversarial Datasets

### Loading adversarial examples

path = ""

with open(path+"New_FGSM_train.pkl", "rb") as f:
  fgsm_train = pkl.load(f)
with open(path+"New_FGSM_test.pkl", "rb") as f:
  fgsm_test = pkl.load(f)

with open(path+"New_PGD_train.pkl", "rb") as f:
  pgd_train = pkl.load(f)
with open(path+"New_PGD_test.pkl", "rb") as f:
  pgd_test = pkl.load(f)

with open(path+"New_DF_train.pkl", "rb") as f:
  cw_train = pkl.load(f)
with open(path+"New_DF_train.pkl", "rb") as f:
  cw_test = pkl.load(f)

with open(path+"New_DF_train.pkl", "rb") as f:
  df_train = pkl.load(f)
with open(path+"New_DF_test.pkl", "rb") as f:
  df_test = pkl.load(f)


### Crafting datasets

x_advs_train_fgsm = []
x_advs_test_fgsm = []
y_advs_train_fgsm = []
y_advs_test_fgsm = []

x_advs_train_pgd = []
x_advs_test_pgd = []
y_advs_train_pgd = []
y_advs_test_pgd = []

x_advs_train_cw = []
x_advs_test_cw = []
y_advs_train_cw = []
y_advs_test_cw = []

x_advs_train_df = []
x_advs_test_df = []
y_advs_train_df = []
y_advs_test_df = []

x_advs_train_balanced = []
x_advs_test_balanced = []
y_advs_train_balanced = []
y_advs_test_balanced = []


for index in range(len(feature_map)):
  labels_0_tr = [0 for _ in range(datasets_train[index].shape[0])]
  labels_0_te = [0 for _ in range(datasets_test[index].shape[0])]
  labels_1_tr = [1 for _ in range(datasets_train[index].shape[0])]
  labels_1_te = [1 for _ in range(datasets_test[index].shape[0])]

  # Clean examples with label 0
  clean_train = pd.DataFrame(datasets_train[index])
  clean_train[clean_train.shape[1]] = labels_0_tr
  clean_test = pd.DataFrame(datasets_test[index])
  clean_test[clean_test.shape[1]] = labels_0_te

  # Altered examples with label 1
  fgsm_train_adv = pd.DataFrame(fgsm_train[index])
  fgsm_train_adv[fgsm_train_adv.shape[1]] = labels_1_tr
  fgsm_test_adv = pd.DataFrame(fgsm_test[index])
  fgsm_test_adv[fgsm_test_adv.shape[1]] = labels_1_te

  pgd_train_adv = pd.DataFrame(pgd_train[index])
  pgd_train_adv[pgd_train_adv.shape[1]] = labels_1_tr
  pgd_test_adv = pd.DataFrame(pgd_test[index])
  pgd_test_adv[pgd_test_adv.shape[1]] = labels_1_te

  cw_train_adv = pd.DataFrame(cw_train[index])
  cw_train_adv[cw_train_adv.shape[1]] = labels_1_tr
  cw_test_adv = pd.DataFrame(cw_test[index])
  cw_test_adv[cw_test_adv.shape[1]] = labels_1_te

  df_train_adv = pd.DataFrame(df_train[index])
  df_train_adv[df_train_adv.shape[1]] = labels_1_tr
  df_test_adv = pd.DataFrame(df_test[index])
  df_test_adv[df_test_adv.shape[1]] = labels_1_te

  size_quarter = clean_train[0].shape[0] // 4
  balanced_train_adv = pd.DataFrame().append([
      fgsm_train_adv.iloc[:size_quarter],
      pgd_train_adv.iloc[size_quarter:2*size_quarter],
      cw_train_adv.iloc[2*size_quarter:3*size_quarter],
      df_train_adv.iloc[3*size_quarter:],
  ])
  size_quarter = clean_test[0].shape[0] // 4
  balanced_test_adv = pd.DataFrame().append([
      fgsm_test_adv.iloc[:size_quarter],
      pgd_test_adv.iloc[size_quarter:2*size_quarter],
      cw_test_adv.iloc[2*size_quarter:3*size_quarter],
      df_test_adv.iloc[3*size_quarter:],
  ])
  
  #to modify the attack dataset
  adversarial_dataset_train = clean_train.append(fgsm_train_adv, ignore_index=True)
  adversarial_dataset_test = clean_test.append(fgsm_test_adv, ignore_index=True)
  x_advs_train_fgsm.append(adversarial_dataset_train.values[:, :-1])
  x_advs_test_fgsm.append(adversarial_dataset_test.values[:, :-1])
  y_advs_train_fgsm.append(adversarial_dataset_train.values[:, -1])
  y_advs_test_fgsm.append(adversarial_dataset_test.values[:, -1])
  
  adversarial_dataset_train = clean_train.append(pgd_train_adv, ignore_index=True)
  adversarial_dataset_test = clean_test.append(pgd_test_adv, ignore_index=True)
  x_advs_train_pgd.append(adversarial_dataset_train.values[:, :-1])
  x_advs_test_pgd.append(adversarial_dataset_test.values[:, :-1])
  y_advs_train_pgd.append(adversarial_dataset_train.values[:, -1])
  y_advs_test_pgd.append(adversarial_dataset_test.values[:, -1])
  
  adversarial_dataset_train = clean_train.append(cw_train_adv, ignore_index=True)
  adversarial_dataset_test = clean_test.append(cw_test_adv, ignore_index=True)
  x_advs_train_cw.append(adversarial_dataset_train.values[:, :-1])
  x_advs_test_cw.append(adversarial_dataset_test.values[:, :-1])
  y_advs_train_cw.append(adversarial_dataset_train.values[:, -1])
  y_advs_test_cw.append(adversarial_dataset_test.values[:, -1])
  
  adversarial_dataset_train = clean_train.append(df_train_adv, ignore_index=True)
  adversarial_dataset_test = clean_test.append(df_test_adv, ignore_index=True)
  x_advs_train_df.append(adversarial_dataset_train.values[:, :-1])
  x_advs_test_df.append(adversarial_dataset_test.values[:, :-1])
  y_advs_train_df.append(adversarial_dataset_train.values[:, -1])
  y_advs_test_df.append(adversarial_dataset_test.values[:, -1])
  
  adversarial_dataset_train = clean_train.append(balanced_train_adv, ignore_index=True)
  adversarial_dataset_test = clean_test.append(balanced_test_adv, ignore_index=True)
  x_advs_train_balanced.append(adversarial_dataset_train.values[:, :-1])
  x_advs_test_balanced.append(adversarial_dataset_test.values[:, :-1])
  y_advs_train_balanced.append(adversarial_dataset_train.values[:, -1])
  y_advs_test_balanced.append(adversarial_dataset_test.values[:, -1])

In [49]:
#%%   begining of the For loop

XadvTrainDataList =[x_advs_train_fgsm,x_advs_train_pgd,x_advs_train_df]
YadvTrainDataList =[y_advs_train_fgsm,y_advs_train_pgd,y_advs_train_df]

XadvTestDataList =[x_advs_test_fgsm,x_advs_test_pgd,x_advs_test_df]
YadvTestDataList =[y_advs_test_fgsm,y_advs_test_pgd,y_advs_test_df]

r = (len(feature_map),len(XadvTrainDataList),len(XadvTestDataList))
ResultsAdD=np.zeros(r)

s = (len(XadvTrainDataList),len(XadvTestDataList))
ResultsFusionMVR=np.zeros(s)
ResultsFusionSBA=np.zeros(s)
ResultsFusionDS=np.zeros(s)
ResultsFusionDSWW=np.zeros(s)

for Itrain ,(x_advs_train,y_advs_train) in enumerate(zip(XadvTrainDataList,YadvTrainDataList)):
    
    for Itest ,(x_advs_test,y_advs_test) in enumerate(zip(XadvTestDataList,YadvTestDataList)):
        print(Itrain,Itest ) 

        
        ## Adversarial Detectors
        
        ### Train ADD
        
        #detectors = ensemble_layer.copy() : this is a shared copy
                
        detectors = []
        for i in range(len(feature_map)):
          detectors.append(tf.keras.models.clone_model(ensemble_layer[i]))
        
        # General parameters
        eval_metric = 'binary_accuracy'
        lr = 0.01
        optim = tf.keras.optimizers.Adam(learning_rate=lr)
        loss_fn = tf.keras.losses.BinaryCrossentropy()
        neurons = 256
        
        rnd=Itrain*100+Itest*10+1
        for detector in detectors: # Add the ReLu layers
          rnd=rnd+1
          detector.pop()
          detector.add(tf.keras.layers.Dense(neurons, activation="relu",name="dense_0001{}".format(rnd)))
          detector.add(tf.keras.layers.Dense(neurons, activation="relu",name="dense_0002{}".format(rnd)))
          detector.add(tf.keras.layers.Dense(1, activation="sigmoid",name="dense_0003{}".format(rnd)))
          detector.compile(optimizer=optim, loss=loss_fn, metrics=[eval_metric]) 
                           
        for index in range(len(detectors)):
          best_value = 0.0
          detector = detectors[index]
          det_file = "./ADD/{}".format(index)
        
          # Iterrating on the dataset
          for i in range(3):
            detector.fit(x=x_advs_train[index], y=y_advs_train[index], epochs=10, batch_size=100, verbose=0)
            eval = detector.evaluate(x_advs_test[index], y_advs_test[index], verbose=0)
            eval_value = eval[-1]
            if eval_value > best_value:
              best_value = eval_value
              tf.keras.models.save_model(detector, det_file)
              print("Detector {} : new best score of {}".format(index, best_value))
        
        ### Evaluate ADD
        
        detectors = []
        for index in range(len(feature_map)):
          detector = tf.keras.models.load_model("./ADD/{}".format(index))
          detectors.append(detector)
          eval = detector.evaluate(x_advs_test[index], y_advs_test[index], verbose=0)[-1]
          ResultsAdD[index][Itrain][Itest]=eval
          print("Detector {} : {}".format(index, eval))
          
        
        
        ## Fusion Rules
        
        ### Definition
        
        #### Majority Vote
        
        def MajorityVoteRule(ClassProbabilities):
          ClassFrequency = np.argmax(ClassProbabilities, axis=1) 
          counts = np.bincount(ClassFrequency)
          MajorityVotingClass = np.argmax(counts)
          return MajorityVotingClass
        
        #### Simple Bayes Average
        
        def SimpleBayesAverageFusion(ClassProbabilities):
          SimpleBayesAverage=sum(ClassProbabilities,0)/ClassProbabilities.shape[0]
          if SimpleBayesAverage[0]>=SimpleBayesAverage[1]:
            return 0,SimpleBayesAverage[0]
          return 1,SimpleBayesAverage[1]
        
        #### Dempster Shafer Evidence Combination
        
        def DempsterRule(m1, m2):
            ## extract the frame of discernment      
            sets=set(m1.keys()).union(set(m2.keys()))
            result=dict.fromkeys(sets,0)
            ## Combination process
            for i in m1.keys():
                for j in m2.keys():
                    if set(str(i)).intersection(set(str(j))) == set(str(i)):
                        result[i]+=m1[i]*m2[j]
                    elif set(str(i)).intersection(set(str(j))) == set(str(j)):
                        result[j]+=m1[i]*m2[j]
                         
            ## normalize the results
            f= sum(list(result.values()))
            #print("f",f)
            for i in result.keys():
                result[i] = result[i]/f
            return result
        
        def DempsterShaferEvidenceCombinationRule (ClassProbabilities, Weights, DontKnowClass=False):
          MassFunctionList= []
          for i in range(ClassProbabilities.shape[0]):
            MassFunction = {"a":Weights[i,0]*ClassProbabilities[i,0], "b":Weights[i,1]*ClassProbabilities[i,1],"ab":1-Weights[i,0]*ClassProbabilities[i,0]-Weights[i,1]*ClassProbabilities[i,1]}
            MassFunctionList.append(MassFunction)
            
          for i,mass in enumerate(MassFunctionList):
            if i == 0:
              DSresult = mass
            else: 
              DSresult = DempsterRule(DSresult,mass)
        
          if DSresult['a'] >= DSresult['b']:
            return 0, DSresult['a']
          else:
            return 1, DSresult['b'] 
        
        def DempsterShaferEvidenceCombinationRuleWithoutWeights (ClassProbabilities,Weights=[0.9999,0.9999]):
          MassFunctionList= []
          for i in range(ClassProbabilities.shape[0]):
            MassFunction = {"a":Weights[0]*ClassProbabilities[i,0], "b":Weights[1]*ClassProbabilities[i,1],"ab":1-Weights[0]*ClassProbabilities[i,0]-Weights[1]*ClassProbabilities[i,1]}
            MassFunctionList.append(MassFunction)
          for i,mass in enumerate(MassFunctionList):
            if i == 0:
              DSresult = mass
            else:
              DSresult = DempsterRule(DSresult,mass)
              
          #print("DSresult apres\n",DSresult)  
          if DSresult['a'] >= DSresult['b']:
            return 0, DSresult['a']
          else:
            return 1, DSresult['b'] 
        ### Compute results
        
        #### Turn mono-neuronal output into bi-class probabilities
        
        probas = []
        for index in range(len(detectors)):
          detector = detectors[index]
          pred = detector.predict(x_advs_test[index])
          proba = []
          for i in pred:
            proba.append([1-i[0], i[0]])
          probas.append(proba)
        
        #### Majority Vote Rule
        
        guesses = []
        for sample in range(len(probas[0])):
          class_proba = np.array([probas[detector][sample] for detector in range(len(detectors))])
          guesses.append(MajorityVoteRule(class_proba))
        
        correct = y_advs_test[0]
        hit = 0
        for i in range(len(guesses)):
          if guesses[i] == correct[i]:
            hit += 1
        acc = hit / len(guesses)
        ResultsFusionMVR[Itrain][Itest]=acc
        print(acc)
        
        #### Simple Bayes Average
        
        guesses = []
        for sample in range(len(probas[0])):
          class_proba = np.array([probas[detector][sample] for detector in range(len(detectors))])
          guesses.append(SimpleBayesAverageFusion(class_proba))
        
        correct = y_advs_test[0]
        hit = 0
        for i in range(len(guesses)):
          if guesses[i][0] == correct[i]:
            hit += 1
        acc = hit / len(guesses)
        ResultsFusionSBA[Itrain][Itest]=acc
        print(acc)
        
        #### Dempster Shafer Evidence Combination
        
        # Get the weights
        weights = []
        correct = y_advs_test[0]
        amount_1 = np.count_nonzero(correct == 1)
        amount_0 = np.count_nonzero(correct == 0)
        for proba in probas:
          weight = []
          hit_0 = 0
          hit_1 = 0
          for i in range(len(proba)):
            if proba[i].index(max(proba[i])) == correct[i]:
              if correct[i]:
                hit_1 += 1
              else:
                hit_0 += 1
          weight = [hit_0/amount_0, hit_1/amount_1]
          weights.append(weight)
        weights = np.array(weights)
        
        guesses = []
        for sample in range(len(probas[0])):
          class_proba = np.array([probas[detector][sample] for detector in range(len(detectors))])
          guesses.append(DempsterShaferEvidenceCombinationRule(class_proba, weights))
        
        correct = y_advs_test[0]
        hit = 0
        for i in range(len(guesses)):
          if guesses[i][0] == correct[i]:
            hit += 1
        acc = hit / len(guesses)
        ResultsFusionDS[Itrain][Itest]=acc
        print(acc)
        
        #### Dempster Shafer Evidence Combination ( without weights)
        
        o=0
        guesses = []
        for sample in range(len(probas[0])):
          o+=1
          #print("o =",o)  
          class_proba = np.array([probas[detector][sample] for detector in range(len(detectors))])
          #print("class_proba =",class_proba) 
          guesses.append(DempsterShaferEvidenceCombinationRuleWithoutWeights(class_proba))
        
        correct = y_advs_test[0]
        hit = 0
        for i in range(len(guesses)):
          if guesses[i][0] == correct[i]:
            hit += 1
        acc = hit / len(guesses)
        ResultsFusionDSWW[Itrain][Itest]=acc
        print(acc)

0 0
INFO:tensorflow:Assets written to: ./ADD/0/assets
Detector 0 : new best score of 0.9994720816612244
INFO:tensorflow:Assets written to: ./ADD/1/assets
Detector 1 : new best score of 0.9960626363754272
INFO:tensorflow:Assets written to: ./ADD/1/assets
Detector 1 : new best score of 0.9962606430053711
INFO:tensorflow:Assets written to: ./ADD/2/assets
Detector 2 : new best score of 0.9979763031005859
INFO:tensorflow:Assets written to: ./ADD/2/assets
Detector 2 : new best score of 0.9981083273887634
INFO:tensorflow:Assets written to: ./ADD/3/assets
Detector 3 : new best score of 0.9680832624435425
INFO:tensorflow:Assets written to: ./ADD/4/assets
Detector 4 : new best score of 0.9405657649040222
Detector 0 : 0.9994720816612244
Detector 1 : 0.9962606430053711
Detector 2 : 0.9981083273887634
Detector 3 : 0.9680832624435425
Detector 4 : 0.9405657649040222
0.9999780036074084
0.9999780036074084
1.0
1.0
0 1
INFO:tensorflow:Assets written to: ./ADD/0/assets
Detector 0 : new best score of 0.991

In [69]:
path=''
ResultsMatrix= np.zeros((5,4))
ensemble_layer=[]
#load Kitsune model
for i in range(len(feature_map)):
  ensemble_layer.append(tf.keras.models.load_model("./Ensemble/{}".format(i)))

output_layer = tf.keras.models.load_model("./Output/")  



with open(path+"clean_test.pkl", "rb") as f: # FGSM
  clean_test = pkl.load(f)
print(" Evaluation the model against clean examples")
ResultsMatrix[0,:]=get_accuracy(clean_test, y_test)


with open(path+"New_FGSM_test.pkl", "rb") as f: # FGSM
  fgsm_dataset_test = pkl.load(f)
print("# Evaluating the model against the adversarial examples FGSM_test")
ResultsMatrix[1,:]=get_accuracy(fgsm_dataset_test, y_test)


with open(path+"New_PGD_test.pkl", "rb") as f: # PGD
  pgd_dataset_test = pkl.load(f)
print("# Evaluating the model against the adversarial examples PGD_test")
ResultsMatrix[2,:]=get_accuracy(pgd_dataset_test, y_test)


with open(path+"New_DF_test.pkl", "rb") as f: # CW
  cw_dataset_test = pkl.load(f)
print("# Evaluating the model against the adversarial examples CW_test")
ResultsMatrix[3,:]=get_accuracy(cw_dataset_test, y_test)

with open(path+"New_DF_test.pkl", "rb") as f: # DF
  df_dataset_test = pkl.load(f)

print("# Evaluating the model against the adversarial examples DF_test")
ResultsMatrix[4,:]=get_accuracy(df_dataset_test, y_test)

print(ResultsMatrix)


 Evaluation the model against clean examples
              precision    recall  f1-score   support

         0.0       0.99      0.98      0.98     11365
         1.0       0.98      0.99      0.98     11366

    accuracy                           0.98     22731
   macro avg       0.98      0.98      0.98     22731
weighted avg       0.98      0.98      0.98     22731

# Evaluating the model against the adversarial examples FGSM_test
              precision    recall  f1-score   support

         0.0       0.44      0.55      0.49     11365
         1.0       0.41      0.31      0.35     11366

    accuracy                           0.43     22731
   macro avg       0.42      0.43      0.42     22731
weighted avg       0.42      0.43      0.42     22731

# Evaluating the model against the adversarial examples PGD_test
              precision    recall  f1-score   support

         0.0       0.17      0.21      0.19     11365
         1.0       0.01      0.00      0.01     11366

    ac

In [70]:
with open("ResultsAdD.pkl", "wb") as f:
  pkl.dump(ResultsAdD, f)
  
with open("ResultsFusionMVR.pkl", "wb") as f:
  pkl.dump(ResultsFusionMVR, f)
with open("ResultsFusionSBA.pkl", "wb") as f:
  pkl.dump(ResultsFusionSBA, f)
with open("ResultsFusionDS.pkl", "wb") as f:
  pkl.dump(ResultsFusionDS, f)
with open("ResultsFusionDSWW.pkl", "wb") as f:
  pkl.dump(ResultsFusionDSWW, f)

with open("ResultsMatrix.pkl", "wb") as f:
  pkl.dump(ResultsMatrix, f)   

In [71]:
print('\n ResultsAdD ')
print(ResultsAdD)


print('\n ResultsFusionMVR ')
print(ResultsFusionMVR)
print('\n ResultsFusionSBA ')
print(ResultsFusionSBA)
print('\n ResultsFusionDS ')
print(ResultsFusionDS)
print('\n ResultsFusionDSWW ')
print(ResultsFusionDSWW)


print('\n ResultsMatrix ')
print(ResultsMatrix)   


 ResultsAdD 
[[[0.99947208 0.99447888 0.76789409]
  [0.99648058 0.99991202 0.76620036]
  [0.89802474 0.91720557 0.92250669]]

 [[0.99626064 0.77057761 0.60087544]
  [0.98853987 0.99571073 0.60014957]
  [0.75269455 0.75152874 0.62043023]]

 [[0.99810833 0.9970085  0.72766268]
  [0.99813032 0.99775636 0.72167963]
  [0.99476486 0.99071753 0.90319389]]

 [[0.96808326 0.51689321 0.52588975]
  [0.5        0.5        0.5       ]
  [0.5        0.5        0.5       ]]

 [[0.94056576 0.89509922 0.61262155]
  [0.92239672 0.94091767 0.60083145]
  [0.92162687 0.86390829 0.85706747]]]

 ResultsFusionMVR 
[[0.999978   0.93126127 0.62533544]
 [0.99793234 0.99989002 0.59761999]
 [0.92855572 0.97210857 0.96467379]]

 ResultsFusionSBA 
[[0.999978   0.9890018  0.61059786]
 [0.99975804 0.999978   0.67550922]
 [0.99683252 0.97228455 0.98031323]]

 ResultsFusionDS 
[[1.         0.99252123 0.50116581]
 [0.99964806 0.999978   0.50378338]
 [0.99929612 0.97564999 0.97967533]]

 ResultsFusionDSWW 
[[1.         0